In [1]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 47.4 MB/s eta 0:00:00


**Business Case:** Supply Chain Optimization for a Food Manufacturing Company
A food manufacturing company produces various food products and distributes them to retailers and wholesalers. The company has several production facilities and warehouses located in different regions, and it also uses third-party logistics (3PL) providers for transportation and storage. The company aims to optimize its supply chain operations to minimize costs and improve delivery times.

**Data and Assumptions**
To perform the analysis, we will need the following data and assumptions:

**Production Facilities**
**Location:** Latitude and Longitude of the production facilities

**Capacity:** Maximum production capacity of each facility

**Fixed Cost:** Fixed cost of operating each facility

**Variable Cost:** Variable cost of producing each unit of product
Warehouses

**Location:** Latitude and Longitude of the warehouses

**Capacity:** Maximum storage capacity of each warehouse

**Fixed Cost:** Fixed cost of operating each warehouse

**Variable Cost:** Variable cost of storing each unit of product
Retailers and Wholesalers(optional)

**Location:** Latitude and Longitude of the retailers and wholesalers

**Demand:** Monthly demand of each product at each retailer/wholesaler

**Cost:** Transportation cost of delivering each unit of product to each retailer/wholesaler

**Link for dataset:** https://www.kaggle.com/datasets/gullayeshwantkumar/data-for-supply-chain-optimization

**Assumptions**
The company produces five different products, each with its own production process and capacity requirements.<br>
The demand for each product is known and constant.<br>
The company operates on a monthly basis.<br>
The transportation cost is calculated based on the distance between the warehouse/production facility and the retailer/wholesaler.<br>
The company can use its own trucks or third-party logistics (3PL) providers for transportation.<br>
The cost of using 3PL providers is higher than using the company's own trucks, but using 3PL providers allows for more flexibility in terms of delivery times and locations.<br>
**Analysis Steps**
To optimize the supply chain operations, we will use the following steps:

1. Define the decision variables:
  1. Quantity of each product to produce at each production facility
  2. Quantity of each product to store at each warehouse
  3. Quantity of each product to deliver to each retailer/wholesaler
2. Define the objective function:
  1. Minimize the total cost of production, storage, and transportation
3. Define the constraints:
  1. Production capacity constraints at each facility
  2. Storage capacity constraints at each warehouse
  3. Demand constraints at each retailer/wholesaler
  4. Non-negative constraints for all decision variables
  5. Solve the optimization model using the MIP (Mixed-Integer Programming) approach
  6. Analyze the results and make recommendations for supply chain improvements.

In [21]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from mip import Model, xsum, minimize, BINARY

# # Define the production facilities data
# facilities_data = pd.DataFrame({
#     'Facility': ['Facility 1', 'Facility 2', 'Facility 3', 'Facility 4'],
#     'Latitude': [51.5074, 51.5074, 52.5200, 52.5200],
#     'Longitude': [-0.1278, -0.1278, 13.4050, 13.4050],
#     'Capacity': [100, 100, 150, 150],
#     'Fixed Cost': [10000, 10000, 15000, 15000]
# })

# # Define the customer demand data
# demand_data = pd.DataFrame({
#     'Customer': ['Customer 1', 'Customer 2', 'Customer 3', 'Customer 4'],
#     'Latitude': [51.5074, 51.5074, 52.5200, 52.5200],
#     'Longitude': [-0.1278, -0.1278, 13.4050, 13.4050],
#     'Demand': [50, 75, 50, 75]
# })
facilities_data=pd.read_csv("facilities_data.csv")
demand_data=pd.read_csv("demand_data.csv")


In [22]:

# Calculate the distances between all facilities and customers
distances = np.zeros((len(facilities_data), len(demand_data)))
for i, facility in facilities_data.iterrows():
    for j, customer in demand_data.iterrows():
        distances[i][j] = geodesic((facility['Latitude'], facility['Longitude']), (customer['Latitude'], customer['Longitude'])).km


In [23]:

# Define the MIP model
model = Model()

# Define the decision variables
x = [[model.add_var(var_type=BINARY) for j in range(len(demand_data))] for i in range(len(facilities_data))]

# Define the objective function
obj = xsum(facilities_data.loc[i, 'Fixed Cost'] * x[i][j] for j in range(len(demand_data)) for i in range(len(facilities_data))) + xsum(distances[i][j] * x[i][j] * demand_data.loc[j, 'Demand'] for j in range(len(demand_data)) for i in range(len(facilities_data)))

model.objective = minimize(obj)


In [24]:

# Define the constraints
for j in range(len(demand_data)):
    model += xsum(x[i][j] for i in range(len(facilities_data))) == 1

for i in range(len(facilities_data)):
    model += xsum(x[i][j] * demand_data.loc[j, 'Demand'] for j in range(len(demand_data))) <= facilities_data.loc[i, 'Capacity']

# Solve the optimization model
model.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [25]:
# Analyze the results
for i in range(len(facilities_data)):
    for j in range(len(demand_data)):
        if x[i][j].x > 0.5:
            print(f"Produce at {facilities_data.loc[i, 'Facility']} for customer {demand_data.loc[j, 'Customer']} with demand {demand_data.loc[j, 'Demand']}")

print(f"Total cost: {model.objective_value}")

Produce at Facility 1 for customer Customer 2 with demand 75
Produce at Facility 2 for customer Customer 1 with demand 50
Produce at Facility 3 for customer Customer 3 with demand 50
Produce at Facility 3 for customer Customer 4 with demand 75
Total cost: 50000.0


This code sets up an optimization problem to minimize the total cost of meeting the demand of four customers from four production facilities located in London and Berlin. The optimization problem takes into account the fixed cost of setting up a production facility, the distance between a production facility and a customer, and the demand of each customer